In [ ]:
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
from tensorflow.keras.utils import to_categorical
import string
import re
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/emotions.txt",header=None,sep=';',names=['Text','Sentiments'])
data

,Text,Sentiments
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
980,im not sure how i feel more than anything im k...,joy
981,i also feel lethargic and again,sadness
982,im feeling somewhat nostalgic about the game j...,love
983,i don t feel like i should be punished to carr...,sadness


In [ ]:
data=data.dropna()
data.shape

(984, 2)

In [ ]:
data['Sentiments'].unique()

array(['sadness', 'joy', 'fear', 'anger', 'love', 'surprise'],
      dtype=object)

In [ ]:
data['Sentiments']=data.Sentiments.replace({'sadness':0,'joy':1,'love':2,'anger':3,'surprise':4,'fear':5})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X=data['Text']

In [ ]:
tokenizer=Tokenizer(15212,lower=True,oov_token='UNK')

In [ ]:
tokenizer.fit_on_texts(X)

In [ ]:
X_train=tokenizer.texts_to_sequences(X)

In [ ]:

X_train_paded=pad_sequences(X_train,maxlen=80,padding='pre')

In [ ]:
Y_train=data['Sentiments'].values

In [ ]:
Y_train_data=to_categorical(Y_train)

In [ ]:
# from sklearn.model_selection import train_test_split
# xtrain,xtest,ytrain,ytest=train_test_split(X_train_paded,Y_train_data,test_size=0.2)
                                          

In [ ]:
#xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((787, 80), (197, 80), (787, 6), (197, 6))

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,LSTM,Bidirectional
from tensorflow.keras.layers import Embedding

In [ ]:
model=Sequential()
#(vocab_size,feature_size,input_shape(maxleng))
model.add(Embedding(15212,64,input_length=80))
model.add(Dropout(0.6))
model.add(Bidirectional(LSTM(80,return_sequences=True)))
model.add(Bidirectional(LSTM(160)))

model.add(Dense(6,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 64)            973568    
                                                                 
 dropout (Dropout)           (None, 80, 64)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 80, 160)          92800     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 320)              410880    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 6)                 1926      
                                                                 
Total params: 1,479,174
Trainable params: 1,479,174
Non-

In [ ]:
model.compile(optimizer=Adam(),loss=CategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
model.fit(X_train_paded,Y_train_data,epochs=10)

Epoch 1/10
31/31 [==============================] - 3s 98ms/step - loss: 0.2806 - accuracy: 0.9106
Epoch 2/10
31/31 [==============================] - 3s 97ms/step - loss: 0.2802 - accuracy: 0.9004
Epoch 3/10
31/31 [==============================] - 3s 99ms/step - loss: 0.2185 - accuracy: 0.9289
Epoch 4/10
31/31 [==============================] - 3s 97ms/step - loss: 0.1693 - accuracy: 0.9370
Epoch 5/10
31/31 [==============================] - 3s 96ms/step - loss: 0.1544 - accuracy: 0.9482
Epoch 6/10
31/31 [==============================] - 3s 97ms/step - loss: 0.1424 - accuracy: 0.9441
Epoch 7/10
31/31 [==============================] - 3s 97ms/step - loss: 0.1119 - accuracy: 0.9654
Epoch 8/10
31/31 [==============================] - 3s 97ms/step - loss: 0.1036 - accuracy: 0.9675
Epoch 9/10
31/31 [==============================] - 3s 98ms/step - loss: 0.1542 - accuracy: 0.9451
Epoch 10/10
31/31 [==============================] - 3s 97ms/step - loss: 0.1166 - accuracy: 0.9634


In [ ]:
def get_key(value):
  dict={'sadness':0,'joy':1,'love':2,'anger':3,'surprise':4,'fear':5}
  for i, j in dict.items():
    if(j==value):
      return i

In [ ]:
def predict(sentence):
  pred_data=[]
  pred_data.append(sentence)
  # print(pred_data)
  sentence_seq=tokenizer.texts_to_sequences(pred_data)
  # print(sentence_seq)
  padded_pred=pad_sequences(sentence_seq,maxlen=80,padding='pre')
  predicted=model.predict(padded_pred)
  results=np.argmax(predicted,axis=1)
  result=get_key(results)
  return result

In [ ]:
from IPython.display import  clear_output
input_sentimentals=print("hi,What is your name ")
user=input()
input_sentimental=input("How are you feeling today ")
result=predict(input_sentimental)
if result=="sadness" or result=='Anger' or result=='fear':
  print("What Happend "+user+'?')
  response=input()
  print("Hey "+user +" I am here for you .Everyone has a hard time in their life at  some point we always hope")
  assistance=input("Do you need any assitance")
  if assistance=='n' or assistance=='no' or assistance=='No' or assistance=='N':
    print("Bye,Bye " + user + " Have a nice Day")
  elif assistance=='y' or assistance=='yes' or assistance=='Y' or assistance=='Yes':
    print("How can i help you")
    print("\n")
    print("1.Do you want to talk with me")
    print("2.Do you want to listen music")
    response=input()
    if response=="1":
      print("tell me "+user)
    elif response=="2":

      print("sa...ri..ga..ma...")
elif result=="sad":
  print("Trying to avoid sadness is trying to avoid life")
  assistance=input("Do you need any assitance")
  if assistance=='n' or assistance=='no' or assistance=='No' or assistance=='N':
    print("Bye,Bye " + user + " HAve a nice Day")
  elif assistance=='y' or assistance=='yes' or assistance=='Y' or assistance=='Yes':
    print("How can i help you")
    print("\n")
    print("1.Do you want talk with me")
    print("2.Do you want to listen music")
    response=input()
    if response=="1":
      print("tell me "+user)
    elif response=="2":

      print("sa...ri..ga..ma...")
else: 
  print("oh " + user + " you are a happy person Bye friend")

hi,What is your name 
amrita
How are you feeling today feel sad
What Happend amrita?
i am breakup
Hey amrita I am here for you .Everyone has a hard time in their life at  some point we always hope
Do you need any assitanceyes
How can i help you


1.Do you want to talk with me
2.Do you want to listen music
1
tell me amrita
